## Practiquemos SQL en Python

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Leemos nuestro dataset como un DataFrame de pandas
df = pd.read_csv('bank_marketing.csv')
df['credit_default'].unique()

array(['no', 'unknown', 'yes'], dtype=object)

#### Primero un poco de EDA

In [2]:
# Dividimos la información en 3 dataframes
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
campaign = df[['client_id', 'contact', 'month', 'day', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]
economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]

# Renombramos columnas
client = client.rename(columns={'client_id': 'id'})
campaign = campaign.rename(columns={
    'duration': 'contact_duration',
    'previous': 'previous_campaign_contacts',
    'y': 'campaign_outcome',
    'poutcome': 'previous_outcome',
    'campaign': 'number_contacts'
})
economics = economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'})

# Limpiamos
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)
client['job'] = client['job'].str.replace('.', '')

# Convertimos "success" y "failure" a columna binaria y "nonexistent" como valores nulos
campaign['previous_outcome'] = campaign['previous_outcome'].map({
    'success': 1,
    'failure': 0,
    'nonexistent': np.nan
})
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'yes': 1, 'no': 0})

# Agregamos una columna llamada campaign_id en campaign
campaign['campaign_id'] = 1

C:\Users\rubos\AppData\Local\Temp\ipykernel_18784\1168402416.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['education'] = client['education'].str.replace('.', '_')
C:\Users\rubos\AppData\Local\Temp\ipykernel_18784\1168402416.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['job'] = client['job'].str.replace('.', '')


In [3]:
# Convertimos la columna month a integers
month_dict = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
campaign['month'] = campaign['month'].map(month_dict)

# Cremos una columna datetime llamada last_contact_date
campaign['last_contact_date'] = pd.to_datetime(2022 * 10000 + campaign['month'] * 100 + campaign['day'], format='%Y%m%d')

# Removemos las varianbles redundantes
campaign = campaign.drop(columns=['contact', 'month', 'day'])

In [4]:
# Guardamos los 3 DataFrames a archivos csv files
# client.to_csv('client.csv', index=False)
# campaign.to_csv('campaign.csv', index=False)
# economics.to_csv('economics.csv', index=False)

In [5]:
# Generamos las tablas para nuestra base de datos en PostgreSQL
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    contact_duration INTEGER,
    number_contacts INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    campaign_id BOOLEAN,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """
CREATE TABLE economics (
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    cons_conf_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

#### Visualizamos nuestros dataframes

In [6]:
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,no,yes,no
41184,41184,46,blue-collar,married,professional_course,no,no,no
41185,41185,56,retired,married,university_degree,no,yes,no
41186,41186,44,technician,married,professional_course,no,no,no


In [7]:
campaign

,client_id,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,261,1,999,0,NaN,0,1,2022-05-13
1,1,149,1,999,0,NaN,0,1,2022-05-19
2,2,226,1,999,0,NaN,0,1,2022-05-23
3,3,151,1,999,0,NaN,0,1,2022-05-27
4,4,307,1,999,0,NaN,0,1,2022-05-03
...,...,...,...,...,...,...,...,...,...
41183,41183,334,1,999,0,NaN,1,1,2022-11-30
41184,41184,383,1,999,0,NaN,0,1,2022-11-06
41185,41185,189,2,999,0,NaN,0,1,2022-11-24
41186,41186,442,1,999,0,NaN,1,1,2022-11-17


In [8]:
economics

,client_id,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_three_months,number_employed
0,0,1.1,93.994,-36.4,4.857,5191.0
1,1,1.1,93.994,-36.4,4.857,5191.0
2,2,1.1,93.994,-36.4,4.857,5191.0
3,3,1.1,93.994,-36.4,4.857,5191.0
4,4,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...
41183,41183,-1.1,94.767,-50.8,1.028,4963.6
41184,41184,-1.1,94.767,-50.8,1.028,4963.6
41185,41185,-1.1,94.767,-50.8,1.028,4963.6
41186,41186,-1.1,94.767,-50.8,1.028,4963.6


#### Trabajemos con Sqlite3 y generemos alguna consultas

In [9]:
# Crear una conexión a la base de datos en memoria
conn = sqlite3.connect(':memory:')

# Cargar los DataFrames 
client.to_sql('client', conn, index=False)
campaign.to_sql('campaign', conn, index=False)
economics.to_sql('economics', conn, index=False)

# Consulta 1: Obtener información sobre los clientes con préstamos
query1 = """
SELECT id, age, job, marital, education
FROM client
WHERE loan = "yes";
"""

# Consulta 2: Contar cuántos contactos de campaña tuvieron un resultado exitoso
query2 = """
SELECT COUNT(*) AS campanias_exitosas
FROM campaign
WHERE campaign_outcome = 1;
"""

# Ejecutar las consultas y mostrar los resultados
result1 = pd.read_sql_query(query1, conn)
result2 = pd.read_sql_query(query2, conn)

print("Resultado de la Consulta 1:")
print(result1)

print("\nResultado de la Consulta 2:")
print(result2)

Resultado de la Consulta 1:
         id  age          job   marital            education
0         4   56     services   married          high_school
1        12   29  blue-collar    single          high_school
2        15   54      retired   married             basic_9y
3        17   46  blue-collar   married             basic_6y
4        18   50  blue-collar   married             basic_9y
...     ...  ...          ...       ...                  ...
6243  41098   29      student    single          high_school
6244  41111   46        admin    single    university_degree
6245  41127   61        admin   married          high_school
6246  41143   29   technician    single  professional_course
6247  41176   38    housemaid  divorced          high_school

[6248 rows x 5 columns]

Resultado de la Consulta 2:
   campanias_exitosas
0                4640


In [10]:
# Podemos convertir nuestros resultados en dataframes
result1 = pd.DataFrame(result1)
result1

,id,age,job,marital,education
0,4,56,services,married,high_school
1,12,29,blue-collar,single,high_school
2,15,54,retired,married,basic_9y
3,17,46,blue-collar,married,basic_6y
4,18,50,blue-collar,married,basic_9y
...,...,...,...,...,...
6243,41098,29,student,single,high_school
6244,41111,46,admin,single,university_degree
6245,41127,61,admin,married,high_school
6246,41143,29,technician,single,professional_course


#### Practiquemos

In [11]:
# Consulta 3: Encontrar los trabajos más comunes entre los clientes:
query3 = """
SELECT job, COUNT(*) AS cantidad
FROM client
GROUP BY job
ORDER BY cantidad DESC;
"""

# Ejecutar la consulta y mostrar los resultados
result3 = pd.read_sql_query(query3, conn)

print("Resultado de la Consulta 3:")
print(result3)

Resultado de la Consulta 3:
              job  cantidad
0           admin     10422
1     blue-collar      9254
2      technician      6743
3        services      3969
4      management      2924
5         retired      1720
6    entrepreneur      1456
7   self-employed      1421
8       housemaid      1060
9      unemployed      1014
10        student       875
11        unknown       330


In [12]:
# Consulta 4: Encontrar el promedio de duración de contacto para cada trabajo:
query4 = """
SELECT c.job, AVG(p.contact_duration) AS avg_contact_duration
FROM client c
JOIN campaign p ON c.id = p.client_id
GROUP BY c.job;
"""

# Ejecutar la consulta y mostrar los resultados
result4 = pd.read_sql_query(query4, conn)

print("Resultado de la Consulta 4:")
print(result4)

Resultado de la Consulta 4:
              job  avg_contact_duration
0           admin            254.312128
1     blue-collar            264.542360
2    entrepreneur            263.267857
3       housemaid            250.454717
4      management            257.058140
5         retired            273.712209
6   self-employed            264.142153
7        services            258.398085
8         student            283.683429
9      technician            250.232241
10     unemployed            249.451677
11        unknown            239.675758


In [13]:
# Consulta 5: Obtener la cantidad promedio de contactos previos en la campaña por nivel educativo:
query5 = """
SELECT c.education, AVG(p.previous_campaign_contacts) AS avg_previous_contacts
FROM client c
JOIN campaign p ON c.id = p.client_id
GROUP BY c.education;
"""

# Ejecutar la consulta y mostrar los resultados
result5 = pd.read_sql_query(query5, conn)

print("Resultado de la Consulta 5:")
print(result5)

Resultado de la Consulta 5:
             education  avg_previous_contacts
0                 None               0.226459
1             basic_4y               0.147031
2             basic_6y               0.129145
3             basic_9y               0.141439
4          high_school               0.185917
5           illiterate               0.111111
6  professional_course               0.163075
7    university_degree               0.192390


In [14]:
# Consulta 6: Mostrar los detalles de los clientes y su información económica asociada:
query6 = """
SELECT c.id, c.age, c.job, c.marital, e.emp_var_rate, e.cons_price_idx, e.cons_conf_idx
FROM client c
JOIN economics e ON c.id = e.client_id;
"""

# Ejecutar la consulta y mostrar los resultados
result6 = pd.read_sql_query(query6, conn)

print("Resultado de la Consulta 6:")
result6 = pd.DataFrame(result6)
result6

Resultado de la Consulta 6:


,id,age,job,marital,emp_var_rate,cons_price_idx,cons_conf_idx
0,0,56,housemaid,married,1.1,93.994,-36.4
1,1,57,services,married,1.1,93.994,-36.4
2,2,37,services,married,1.1,93.994,-36.4
3,3,40,admin,married,1.1,93.994,-36.4
4,4,56,services,married,1.1,93.994,-36.4
...,...,...,...,...,...,...,...
41183,41183,73,retired,married,-1.1,94.767,-50.8
41184,41184,46,blue-collar,married,-1.1,94.767,-50.8
41185,41185,56,retired,married,-1.1,94.767,-50.8
41186,41186,44,technician,married,-1.1,94.767,-50.8


In [15]:
# Consulta 7: Contar cuántos clientes tienen credit_default "unknown" y han tenido contacto previo en la campaña:
query7 = """
SELECT COUNT(*) AS cantidad_clientes
FROM client c
JOIN campaign p ON c.id = p.client_id
WHERE c.credit_default = "unknown" AND p.previous_campaign_contacts > 0;
"""

# Ejecutar la consulta y mostrar los resultados
result7 = pd.read_sql_query(query7, conn)

print("Resultado de la Consulta 7:")
print(result7)

Resultado de la Consulta 7:
   cantidad_clientes
0                575


In [16]:
# Consulta 8: Obtener el número de contactos promedio según el día de último contacto:
query8 = """
SELECT strftime('%w', p.last_contact_date) AS dia_de_la_semana, AVG(p.number_contacts) AS avg_number_contacts
FROM campaign p
GROUP BY dia_de_la_semana
ORDER BY dia_de_la_semana;
"""

# Ejecutar la consulta y mostrar los resultados
result8 = pd.read_sql_query(query8, conn)

print("Resultado de la Consulta 8:")
print(result8)

Resultado de la Consulta 8:
  dia_de_la_semana  avg_number_contacts
0                0             2.596466
1                1             2.547929
2                2             2.495648
3                3             2.583434
4                4             2.600748
5                5             2.544259
6                6             2.607597


In [17]:
# Consulta 9: Encontrar el trabajo más común entre los clientes que han tenido éxito en la campaña:
query9 = """
SELECT c.job AS trabajo_mas_comun, COUNT(*) AS apariciones
FROM client c
JOIN campaign p ON c.id = p.client_id
WHERE p.campaign_outcome = 1
GROUP BY c.job
ORDER BY apariciones DESC
LIMIT 1;
"""

# Ejecutar la consulta y mostrar los resultados
result9 = pd.read_sql_query(query9, conn)

print("Resultado de la Consulta 9:")
print(result9)

Resultado de la Consulta 9:
  trabajo_mas_comun  apariciones
0             admin         1352


In [18]:
# Consulta 9: Mostrar la información económica promedio de los clientes casados:
query10 = """
SELECT last_contact_date, AVG(contact_duration) AS avg_contact_duration
FROM campaign
WHERE previous_outcome = 1
GROUP BY last_contact_date
HAVING COUNT(*) >= 10;
"""

# Ejecutar la consulta y mostrar los resultados
result10 = pd.read_sql_query(query10, conn)

print("Resultado de la Consulta 10:")
print(result10)

Resultado de la Consulta 10:
      last_contact_date  avg_contact_duration
0   2022-04-08 00:00:00            314.800000
1   2022-05-02 00:00:00            230.272727
2   2022-05-03 00:00:00            291.272727
3   2022-05-04 00:00:00            502.700000
4   2022-05-06 00:00:00            694.500000
5   2022-05-10 00:00:00            280.928571
6   2022-05-16 00:00:00            325.769231
7   2022-05-30 00:00:00            360.700000
8   2022-06-16 00:00:00            409.500000
9   2022-10-28 00:00:00            261.272727
10  2022-11-01 00:00:00            367.818182


#### ¡Nunca nos olvidemos de cerrar la conexión a la base de datos utilizada!

In [20]:
# Último paso: cerrar la conexión a la base de datos
conn.close()